# **Project Name**    -  **Local Food Wastage Management System**



##### **Project Type**    - EDA
##### **Contribution**    - Individual
##### **Team Member 1 -**  Adarsha Kumar Karna


# **Project Summary -**


**Project Summary: Local Food Wastage Management System**

The Local Food Wastage Management System project was initiated to address the critical, dual-sided issue of food surplus and food insecurity. The primary objective was to develop a data-driven platform where individuals and restaurants can list excess food, and local NGOs or those in need can claim it. The project successfully navigated through a complete development lifecycle, from raw data processing and database engineering to in-depth analysis and the creation of an interactive web application.

The technological foundation of the project was built within the versatile Python ecosystem, utilizing Google Colab as the primary development environment. The core of the data storage is a robust SQL database. While initial explorations considered a tunneled local MySQL setup, a more agile and self-contained SQLite database was ultimately implemented, proving ideal for portability and seamless integration with Colab. Data interaction was managed by the powerful SQLAlchemy library, with the pandas library serving as the backbone for all data manipulation tasks.

A significant portion of the project was dedicated to building a resilient ETL (Extract, Transform, Load) pipeline. Raw data, provided in four separate CSV files—Providers, Receivers, Food Listings, and Claims—was first extracted into pandas DataFrames. The transformation phase was critical and involved intensive data cleaning to ensure schema consistency. This included renaming columns to match the SQL table design, combining multiple fields (e.g., 'Address' and 'City') into a single 'location' column, and selecting only relevant data for insertion. This meticulous process resolved numerous data loading errors, culminating in a rerunnable script that clears and repopulates the database, ensuring data integrity during development.

With the database successfully populated, the project moved into an exploratory data analysis (EDA) phase to uncover actionable insights. Using SQL queries and Python's Matplotlib and Seaborn libraries, a suite of visualizations was generated. Bar charts revealed the top 10 most active food providers, the most common food items being donated, and the geographical hotspots for donations. A pie chart was created to visualize the crucial ratio of "available" versus "claimed" listings, serving as a key performance indicator for the system's effectiveness. Further analysis explored donation quantity distributions and claim activity by day of the week. A key analytical achievement was the creation of a heatmap, which cross-referenced food items with days of the week to identify patterns in when specific types of food are claimed.

The final phase involved developing a user-facing application using Streamlit. This interactive dashboard serves as the central hub for the system. It features a sidebar with dynamic filters, allowing users to browse available food listings and narrow them down by location. The dashboard displays the key visualizations from the analysis phase, providing an at-a-glance overview of the system's activity. Crucially, the application implements the "Create" part of CRUD (Create, Read, Update, Delete) functionality through an intuitive web form, enabling users to add new food donations directly to the live database. This successful integration of a robust backend database with a user-friendly frontend marks the completion of a significant project milestone, laying the groundwork for full deployment and real-world impact.

# **GitHub Link -**

Provide your GitHub Link here.

https://github.com/adrs01

# **Problem Statement**


**Write Problem Statement Here.**

Food wastage from local restaurants and households coexists with food insecurity in the same communities. This project addresses the critical logistical gap by developing a platform to connect food donors directly with receivers in real-time, aiming to reduce waste and efficiently distribute surplus food to those in need.

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

#**Mount Google Drive**

In [ ]:
# Mount Google Drive to save your database file
from google.colab import drive
drive.mount('/content/drive')

#***1.DATA PREPRATION AND DATABASE CREATION:-***

**-creating table**

**-load the datasets into pandas**

**-data preparation/cleaning**

**-insert data into tables**

**-verify tables**

#**Create a Connection to a SQLite database file**

In [ ]:
from sqlalchemy import create_engine

db_path = '/content/drive/My Drive/Colab Notebooks/food_wastage.db'
engine = create_engine(f'sqlite:///{db_path}')

try:
    connection = engine.connect()
    print(" Successfully created/connected to the SQLite database file!")
except Exception as e:
    print(f" An error occurred: {e}")


#**Creating Table**

In [ ]:
# SQL statements to create the tables

from sqlalchemy import text

create_providers_table_sql = """
CREATE TABLE IF NOT EXISTS providers (
    provider_id INTEGER PRIMARY KEY AUTOINCREMENT,
    provider_name VARCHAR(255) NOT NULL,
    location VARCHAR(255),
    contact_info VARCHAR(255)
);
"""

create_receivers_table_sql = """
CREATE TABLE IF NOT EXISTS receivers (
    receiver_id INTEGER PRIMARY KEY AUTOINCREMENT,
    receiver_name VARCHAR(255) NOT NULL,
    type VARCHAR(50),
    location VARCHAR(255)
);
"""

create_food_listings_table_sql = """
CREATE TABLE IF NOT EXISTS food_listings (
    listing_id INTEGER PRIMARY KEY AUTOINCREMENT,
    provider_id INTEGER,
    food_item VARCHAR(255) NOT NULL,
    quantity INTEGER,
    expiration_date DATE,
    status VARCHAR(50) DEFAULT 'available',
    FOREIGN KEY (provider_id) REFERENCES providers(provider_id)
);
"""

create_claims_table_sql = """
CREATE TABLE IF NOT EXISTS claims (
    claim_id INTEGER PRIMARY KEY AUTOINCREMENT,
    listing_id INTEGER,
    receiver_id INTEGER,
    claim_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (listing_id) REFERENCES food_listings(listing_id),
    FOREIGN KEY (receiver_id) REFERENCES receivers(receiver_id)
);
"""

# --- Execute the CREATE TABLE statements ---
# The 'connection' object is from our previous cell
try:
    # Wrap each SQL string in the text() function
    connection.execute(text(create_providers_table_sql))
    connection.execute(text(create_receivers_table_sql))
    connection.execute(text(create_food_listings_table_sql))
    connection.execute(text(create_claims_table_sql))

    # Commit the changes to make them permanent in the database file
    connection.commit()

    print(" All tables created successfully (if they didn't already exist).")
except Exception as e:
    print(f" An error occurred during table creation: {e}")


#**Load the datasets into pandas**

In [ ]:
import pandas as pd
from google.colab import drive

providers_csv_path = '/content/drive/My Drive/Food waste management project/food waste management datasets/providers_data.csv'
receivers_csv_path = '/content/drive/My Drive/Food waste management project/food waste management datasets/receivers_data.csv'
food_listings_csv_path = '/content/drive/My Drive/Food waste management project/food waste management datasets/food_listings_data.csv'
claims_csv_path = '/content/drive/My Drive/Food waste management project/food waste management datasets/claims_data.csv'

# Load CSVs into Pandas DataFrames
providers_df = pd.read_csv(providers_csv_path)
receivers_df = pd.read_csv(receivers_csv_path)
food_listings_df = pd.read_csv(food_listings_csv_path)
claims_df = pd.read_csv(claims_csv_path)

print("Providers Data:")
print(providers_df.head())

##**Data preparation cell**

In [ ]:
# --- 1. Prepare the Providers DataFrame ---
providers_df_renamed = providers_df.rename(columns={
    'Provider_ID': 'provider_id', 'Name': 'provider_name', 'Contact': 'contact_info'
})
providers_df_renamed['location'] = providers_df_renamed['Address'] + ', ' + providers_df_renamed['City']
providers_to_sql = providers_df_renamed[['provider_id', 'provider_name', 'location', 'contact_info']]
print(" Providers DataFrame is ready.")

# --- 2. Prepare the Receivers DataFrame ---
receivers_df_renamed = receivers_df.rename(columns={
    'Receiver_ID': 'receiver_id', 'Name': 'receiver_name', 'Type': 'type'
})
receivers_df_renamed['location'] = receivers_df_renamed['City']
receivers_to_sql = receivers_df_renamed[['receiver_id', 'receiver_name', 'type', 'location']]
print(" Receivers DataFrame is ready.")

# --- 3. Prepare the Food Listings DataFrame  ---
print("\nOriginal Food Listings Columns:", food_listings_df.columns)
food_listings_renamed = food_listings_df.rename(columns={
    'Food_ID': 'listing_id',
    'Food_Name': 'food_item',
    'Quantity': 'quantity',
    'Expiry_Date': 'expiration_date',
    'Provider_ID': 'provider_id'
})
# Select only the columns that exist in the 'food_listings' SQL table
food_listings_to_sql = food_listings_renamed[['listing_id', 'provider_id', 'food_item', 'quantity', 'expiration_date']]
print(" Food Listings DataFrame is ready.")


# --- 4. Prepare the Claims DataFrame  ---
print("\nOriginal Claims Columns:", claims_df.columns)
claims_renamed = claims_df.rename(columns={
    'Claim_ID': 'claim_id',
    'Food_ID': 'listing_id',          # Corrected column name
    'Receiver_ID': 'receiver_id',
    'Timestamp': 'claim_date'       # Corrected column name
})
# Select only the columns that exist in the 'claims' SQL table
claims_to_sql = claims_renamed[['claim_id', 'listing_id', 'receiver_id', 'claim_date']] # Corrected column names for selection
print(" Claims DataFrame is ready.")

#**Insert Data into the Tables**

In [ ]:
from sqlalchemy import text

# The 'engine' object is from our first cell
try:
    # Clear all existing data from the tables first
    with engine.connect() as connection:
        connection.execute(text("DELETE FROM claims"))
        connection.execute(text("DELETE FROM food_listings"))
        connection.execute(text("DELETE FROM receivers"))
        connection.execute(text("DELETE FROM providers"))
        connection.commit()
    print(" All tables cleared successfully.")

    # Now, insert the clean data
    providers_to_sql.to_sql('providers', con=engine, if_exists='append', index=False)
    receivers_to_sql.to_sql('receivers', con=engine, if_exists='append', index=False)


    food_listings_to_sql.to_sql('food_listings', con=engine, if_exists='append', index=False)
    claims_to_sql.to_sql('claims', con=engine, if_exists='append', index=False)

    print(" Data loaded into all tables successfully!")

except Exception as e:
    print(f" An error occurred during data loading: {e}")

#**verify the data**

In [ ]:
import pandas as pd
print("--- Verifying Data in All Tables ---")

try:
    # Query the 'providers' table
    providers_verify_df = pd.read_sql("SELECT * FROM providers LIMIT 5", engine)
    print("\n Top 5 rows from the 'providers' table:")
    print(providers_verify_df)

    # Query the 'receivers' table
    receivers_verify_df = pd.read_sql("SELECT * FROM receivers LIMIT 5", engine)
    print("\n Top 5 rows from the 'receivers' table:")
    print(receivers_verify_df)

    # Query the 'food_listings' table
    listings_verify_df = pd.read_sql("SELECT * FROM food_listings LIMIT 5", engine)
    print("\n Top 5 rows from the 'food_listings' table:")
    print(listings_verify_df)

    # Query the 'claims' table
    claims_verify_df = pd.read_sql("SELECT * FROM claims LIMIT 5", engine)
    print("\n Top 5 rows from the 'claims' table:")
    print(claims_verify_df)

except Exception as e:
    print(f"\n An error occurred while verifying the data: {e}")

#***2.DATA ANALYSIS:-***

**Analysis 1: What are the most commonly donated food items?**

In [ ]:
# SQL query to count the occurrences of each food item
query1 = """
SELECT
    food_item,
    COUNT(listing_id) AS number_of_listings
FROM
    food_listings
GROUP BY
    food_item
ORDER BY
    number_of_listings DESC
LIMIT 10;
"""

# Execute the query and load the result into a DataFrame
top_foods_df = pd.read_sql(query1, engine)

# --- Visualization ---
plt.figure(figsize=(12, 7))
sns.barplot(x='number_of_listings', y='food_item', data=top_foods_df, palette='viridis')
plt.title('Top 10 Most Commonly Donated Food Items')
plt.xlabel('Number of Listings')
plt.ylabel('Food Item')
plt.show()

**Analysis 2: Which providers are the most active?**

In [ ]:
# SQL query to join providers and listings, then count listings per provider
query2 = """
SELECT
    p.provider_name,
    COUNT(fl.listing_id) AS total_listings
FROM
    providers p
JOIN
    food_listings fl ON p.provider_id = fl.provider_id
GROUP BY
    p.provider_name
ORDER BY
    total_listings DESC
LIMIT 10;
"""

# Execute the query
top_providers_df = pd.read_sql(query2, engine)

# --- Visualization ---
plt.figure(figsize=(12, 7))
sns.barplot(x='total_listings', y='provider_name', data=top_providers_df, palette='plasma')
plt.title('Top 10 Most Active Food Providers')
plt.xlabel('Total Number of Food Listings')
plt.ylabel('Provider Name')
plt.show()

**Analysis 3: Which locations have the most food listings?**

In [ ]:
# SQL query to count listings by city/location
query3 = """
SELECT
    p.location,
    COUNT(fl.listing_id) AS number_of_listings
FROM
    providers p
JOIN
    food_listings fl ON p.provider_id = fl.provider_id
GROUP BY
    p.location
ORDER BY
    number_of_listings DESC
LIMIT 10;
"""

# Execute the query
top_locations_df = pd.read_sql(query3, engine)

# --- Visualization ---
plt.figure(figsize=(12, 7))
sns.barplot(x='number_of_listings', y='location', data=top_locations_df, palette='magma')
plt.title('Top 10 Locations by Food Listings')
plt.xlabel('Number of Listings')
plt.ylabel('Location')
plt.show()

**Analysis 4: How many listings are claimed vs. available?**

In [ ]:
query4 = """
SELECT
    status,
    COUNT(listing_id) as count
FROM
    food_listings
GROUP BY
    status;
"""

status_df = pd.read_sql(query4, engine)

# --- Visualization ---
plt.figure(figsize=(8, 8))
plt.pie(status_df['count'], labels=status_df['status'], autopct='%1.1f%%', startangle=140, colors=['#FF9999','#66B2FF'])
plt.title('Proportion of Food Listings by Status (Claimed vs. Available)')
plt.ylabel('') # Hides the 'count' label on the y-axis
plt.show()

**Analysis 5: What is the claim activity by day of the week?**

In [ ]:
# Query to extract the day of the week from the claim_date and count claims
# Note: 'W' is for day of week. 0=Sunday, 1=Monday, ...
query5 = """
SELECT
    STRFTIME('%w', claim_date) as day_of_week_num,
    CASE STRFTIME('%w', claim_date)
        WHEN '0' THEN 'Sunday'
        WHEN '1' THEN 'Monday'
        WHEN '2' THEN 'Tuesday'
        WHEN '3' THEN 'Wednesday'
        WHEN '4' THEN 'Thursday'
        WHEN '5' THEN 'Friday'
        ELSE 'Saturday'
    END as day_of_week_name,
    COUNT(claim_id) as number_of_claims
FROM
    claims
GROUP BY
    day_of_week_num
ORDER BY
    day_of_week_num;
"""
claims_by_day_df = pd.read_sql(query5, engine)

# --- Visualization ---
plt.figure(figsize=(12, 7))
sns.barplot(x='day_of_week_name', y='number_of_claims', data=claims_by_day_df, palette='cubehelix')
plt.title('Food Claim Activity by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Number of Claims')
plt.show()

**Analysis 6: What is the distribution of donation quantities?**

In [ ]:
# Query to get all the quantity data
query6 = "SELECT quantity FROM food_listings;"
quantity_df = pd.read_sql(query6, engine)

# --- Visualization ---
plt.figure(figsize=(12, 7))
# We will filter out extreme outliers for a more readable chart, e.g., quantity < 200
sns.histplot(quantity_df[quantity_df['quantity'] < 200]['quantity'], bins=30, kde=True)
plt.title('Distribution of Food Donation Quantities')
plt.xlabel('Quantity')
plt.ylabel('Frequency')
plt.show()

**Analysis 7: Heatmap of Popular Food Items by Day of Week**

In [ ]:
# A more complex query to join tables and group by two variables
query7 = """
SELECT
    fl.food_item,
    CASE STRFTIME('%w', c.claim_date)
        WHEN '0' THEN 'Sun'
        WHEN '1' THEN 'Mon'
        WHEN '2' THEN 'Tue'
        WHEN '3' THEN 'Wed'
        WHEN '4' THEN 'Thu'
        WHEN '5' THEN 'Fri'
        ELSE 'Sat'
    END as day_of_week,
    COUNT(c.claim_id) as claim_count
FROM
    claims c
JOIN
    food_listings fl ON c.listing_id = fl.listing_id
GROUP BY
    fl.food_item, day_of_week
"""
heatmap_df = pd.read_sql(query7, engine)

# We need to pivot the data to create a matrix for the heatmap
# We'll focus on the top 10 most claimed food items for clarity
top_10_foods = heatmap_df.groupby('food_item')['claim_count'].sum().nlargest(10).index
heatmap_df_filtered = heatmap_df[heatmap_df['food_item'].isin(top_10_foods)]
heatmap_pivot = heatmap_df_filtered.pivot_table(index='food_item', columns='day_of_week', values='claim_count', fill_value=0)

# Reorder columns for logical day progression
day_order = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
heatmap_pivot = heatmap_pivot.reindex(columns=day_order)


# --- Visualization ---
plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_pivot, annot=True, fmt="f", cmap="YlGnBu", linewidths=.5)
plt.title('Heatmap of Claim Frequency: Food Item vs. Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Food Item')
plt.show()

#***3.APPLICATION DEVELOPMENT:-***

**-Install Streamlit and pyngrok**

In [ ]:
!pip install streamlit pyngrok sqlalchemy pandas

#**-Create a python file for streamlit app**

In [ ]:
# %%writefile app.py
# import streamlit as st
# import pandas as pd
# from sqlalchemy import create_engine, text

# # --- Database Connection ---
# # Use the same path to your SQLite database file as before
# db_path = '/content/drive/My Drive/Colab Notebooks/food_wastage.db'
# engine = create_engine(f'sqlite:///{db_path}')

# # --- Page Configuration ---
# st.set_page_config(
#     page_title="Food Wastage Management Dashboard",
#     page_icon="🍔",
#     layout="wide"
# )

# # --- Main App Layout ---
# st.title("Local Food Wastage Management System")
# st.write("This dashboard helps manage and analyze food donation data.")


# # --- Section 1: Browse Raw Data ---
# st.header("Browse Raw Data Tables")
# selected_table = st.selectbox("Choose a table to view", ["providers", "receivers", "food_listings", "claims"])

# if selected_table:
#     try:
#         query = f"SELECT * FROM {selected_table};"
#         df = pd.read_sql(query, engine)
#         st.dataframe(df)
#     except Exception as e:
#         st.error(f"An error occurred: {e}")


# # --- Section 2: Data Analysis ---
# st.header("Data Analysis and Insights")

# # Let's add the "Top 10 Most Active Providers" chart from our analysis
# st.subheader("Top 10 Most Active Food Providers")
# query_top_providers = """
#     SELECT
#         p.provider_name,
#         COUNT(fl.listing_id) AS total_listings
#     FROM
#         providers p
#     JOIN
#         food_listings fl ON p.provider_id = fl.provider_id
#     GROUP BY
#         p.provider_name
#     ORDER BY
#         total_listings DESC
#     LIMIT 10;
# """
# try:
#     top_providers_df = pd.read_sql(query_top_providers, engine)

#     # Using st.bar_chart which is simple and effective
#     # The dataframe needs to have the label column set as the index
#     st.bar_chart(top_providers_df.set_index('provider_name'))

# except Exception as e:
#     st.error(f"An error occurred while generating the chart: {e}")

#**Run the App and Launch ngrok**

In [ ]:
# from pyngrok import ngrok

# # Terminate any existing ngrok tunnels to avoid errors
# ngrok.kill()

# # Paste your ngrok authtoken here
# NGROK_AUTH_TOKEN = "30h8lGVso50GSJoQ1OZ3TfAKIl3_2AEHhZukQCnGFBTVfY7fV"  # <--- PASTE YOUR TOKEN HERE
# ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# # Open a public tunnel to the Streamlit port (8501)
# public_url = ngrok.connect(8501)
# print(f" Your Streamlit app is live! URL: {public_url}")

# # Run the Streamlit app in the background
# # The '--server.runOnSave true' flag will automatically update the app when you change app.py
# !streamlit run app.py --server.runOnSave true &

#**-Creating/updating the exsiting python file of streamlit app for CRUD operations**

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

# --- Database Connection ---
# Use the same path to your SQLite database file
db_path = '/content/drive/My Drive/Colab Notebooks/food_wastage.db'
engine = create_engine(f'sqlite:///{db_path}')

# --- Page Configuration ---
st.set_page_config(
    page_title="Food Wastage Management Dashboard",
    page_icon="🍔",
    layout="wide"
)

# --- Main App Layout ---
st.title("Local Food Wastage Management System")
st.write("This dashboard helps manage and analyze food donation data.")

# --- FEATURE 1: Interactive Filtering via Sidebar ---

st.sidebar.header("Filter Options")

# Fetch all unique locations from the providers table for the filter
with engine.connect() as connection:
    locations_query = text("SELECT DISTINCT location FROM providers ORDER BY location;")
    locations = connection.execute(locations_query).fetchall()
    # The result is a list of tuples, so we extract the first element of each tuple
    location_list = ["All"] + [location[0] for location in locations]

selected_location = st.sidebar.selectbox("Filter by Location", location_list)

# --- Display Filtered Food Listings (This is a "Read" operation) ---
st.header("Available Food Listings")
st.write("Use the filter on the left to narrow down results.")

# Base query for food listings
listings_query = """
    SELECT
        fl.food_item,
        fl.quantity,
        fl.expiration_date,
        p.provider_name,
        p.location,
        p.contact_info
    FROM
        food_listings fl
    JOIN
        providers p ON fl.provider_id = p.provider_id
    WHERE
        fl.status = 'available'
"""

# If a specific location is selected (and not 'All'), add a WHERE clause
if selected_location != "All":
    listings_query += " AND p.location = :location"
    params = {"location": selected_location}
    filtered_listings_df = pd.read_sql(text(listings_query), engine, params=params)
else:
    filtered_listings_df = pd.read_sql(text(listings_query), engine)

st.dataframe(filtered_listings_df)

# --- FEATURE 2: Add a New Food Listing (This is a "Create" operation) ---

st.header("Add a New Food Donation")

# Fetch provider names for the dropdown menu
with engine.connect() as connection:
    providers_query = text("SELECT provider_id, provider_name FROM providers ORDER BY provider_name;")
    providers = connection.execute(providers_query).fetchall()
    provider_dict = {name: pid for pid, name in providers} # Create a name-to-ID dictionary

# Use a form to group inputs. The app won't rerun until the 'Submit' button is clicked.
with st.form("new_listing_form"):
    selected_provider_name = st.selectbox("Select the Provider", options=list(provider_dict.keys()))
    food_item = st.text_input("Food Item Name")
    quantity = st.number_input("Quantity", min_value=1)
    expiration_date = st.date_input("Expiration Date")

    submitted = st.form_submit_button("Add Listing")

    if submitted:
        provider_id = provider_dict[selected_provider_name] # Get the ID from the selected name
        insert_query = text("""
            INSERT INTO food_listings (provider_id, food_item, quantity, expiration_date, status)
            VALUES (:provider_id, :food_item, :quantity, :expiration_date, 'available')
        """)

        try:
            with engine.connect() as connection:
                connection.execute(insert_query, {
                    "provider_id": provider_id,
                    "food_item": food_item,
                    "quantity": quantity,
                    "expiration_date": expiration_date.strftime('%Y-%m-%d')
                })
                connection.commit()
            st.success(" New listing added successfully!")
            # st.rerun() # Optional: Uncomment to automatically refresh the app state
        except Exception as e:
            st.error(f"An error occurred: {e}")

# --- Data Analysis Section ---

st.header("Data Analysis and Insights")
col1, col2 = st.columns(2)

with col1:
    st.subheader("Top 10 Most Active Providers")
    query_top_providers = text("""
        SELECT p.provider_name, COUNT(fl.listing_id) AS total_listings
        FROM providers p JOIN food_listings fl ON p.provider_id = fl.provider_id
        GROUP BY p.provider_name ORDER BY total_listings DESC LIMIT 10;
    """)
    top_providers_df = pd.read_sql(query_top_providers, engine)
    st.bar_chart(top_providers_df.set_index('provider_name'))

with col2:
    # --- NEW FEATURE 3: Added Pie Chart Visualization ---
    st.subheader("Listings by Status")
    query_status = text("""
        SELECT status, COUNT(listing_id) as count FROM food_listings GROUP BY status;
    """)
    status_df = pd.read_sql(query_status, engine)

    fig, ax = plt.subplots()
    ax.pie(status_df['count'], labels=status_df['status'], autopct='%1.1f%%', startangle=90, colors=['#FF9999','#66B2FF'])
    ax.axis('equal') # Equal aspect ratio ensures that pie is drawn as a circle.
    st.pyplot(fig)

#**Run the App and Launch ngrok**

In [ ]:
from pyngrok import ngrok

# Terminate any existing ngrok tunnels to avoid errors
ngrok.kill()

# Paste your ngrok authtoken here
NGROK_AUTH_TOKEN = "30h8lGVso50GSJoQ1OZ3TfAKIl3_2AEHhZukQCnGFBTVfY7fV"  # <--- PASTE YOUR TOKEN HERE
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open a public tunnel to the Streamlit port (8501)
public_url = ngrok.connect(8501)
print(f" Your Streamlit app is live! URL: {public_url}")

# Run the Streamlit app in the background
# The '--server.runOnSave true' flag will automatically update the app when you change app.py
!streamlit run app.py --server.runOnSave true &

# **Conclusion**

This project successfully developed a functional prototype of the Local Food Wastage Management System, proving the concept's viability. By processing raw data into a structured SQLite database and building an interactive Streamlit application, we created a tool that effectively visualizes key trends, allows users to filter available food listings, and implements essential "Create" and "Read" functionality. The system serves as a robust foundation for a real-world solution, with clear next steps toward implementing full CRUD operations and cloud deployment to actively combat local food waste.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***